## LangChain tools

In [5]:
!pip install -q langchain langchain-openai python-dotenv

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Proste narzędzie (kalkulator)

In [6]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Mnoży dwie liczby całkowite."""
    return x * y

print(multiply_numbers.invoke({"x": 6, "y": 7}))


42


### Użycie narzędzia z LLM

In [7]:
from langchain.agents import initialize_agent, AgentType

tools = [multiply_numbers]
#własne API jako tool
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Model sam zdecyduje, że musi użyć multiply_numbers
agent.run("Policz 23 * 7")


/tmp/ipykernel_7758/3603958280.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_7758/3603958280.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Policz 23 * 7")




> Entering new AgentExecutor chain...
Thought: I need to multiply the numbers 23 and 7 to find the answer.
Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "x": 23,
    "y": 7
  }
}
```

Observation: 161
Thought:I know what to respond. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "23 * 7 = 161"
}
```


> Finished chain.


'23 * 7 = 161'

### Kolejne narzędzie (prosta baza wiedzy)

In [8]:
@tool
def company_info(name: str) -> str:
    """Zwraca podstawowe informacje o firmie na podstawie nazwy."""
    companies = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT.",
        "LangChain": "Biblioteka ułatwiająca budowanie aplikacji z modelami językowymi.",
        "Tesla": "Producent samochodów elektrycznych i baterii."
    }
    return companies.get(name, "Nie znam tej firmy.")

tools = [company_info]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Powiedz mi, czym zajmuje się firma Tesla.")




> Entering new AgentExecutor chain...
Muszę uzyskać podstawowe informacje o firmie Tesla, aby dowiedzieć się, czym się zajmuje.  
Action: company_info  
Action Input: "Tesla"  
Observation: Producent samochodów elektrycznych i baterii.
Thought:I now know the final answer  
Final Answer: Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.

> Finished chain.


'Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.'

### Własne API jako tool

In [9]:
import requests

@tool
def get_exchange_rate(currency: str) -> str:
    """Zwraca kurs wymiany waluty względem USD."""
    #set API key
    url = f"https://api.exchangerate.host/latest?base=USD&symbols={currency.upper()}"
    response = requests.get(url).json()
    print(response)
    rate = response["rates"].get(currency.upper())
    return f"1 USD = {rate} {currency.upper()}"

tools = [multiply_numbers, company_info, get_exchange_rate]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Jaki jest aktualny kurs euro względem dolara?")




> Entering new AgentExecutor chain...
Thought: I need to find the current exchange rate of the euro (EUR) against the US dollar (USD). 
Action:
```
{
  "action": "get_exchange_rate",
  "action_input": {
    "currency": "EUR"
  }
}
```
{'success': False, 'error': {'code': 101, 'type': 'missing_access_key', 'info': 'You have not supplied an API Access Key. [Required format: access_key=YOUR_ACCESS_KEY]'}}


KeyError: 'rates'